In [1]:
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split



In [4]:
df = pd.read_csv("../data/raw/HAM10000_metadata.csv")

df.dx.value_counts()

dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64

División de imagenes en malignas y benignas

In [8]:
# Leer el CSV
df = pd.read_csv("../data/raw/HAM10000_metadata.csv")

# Definir clases benignas
clases_benignas = {'nv', 'bkl', 'df', 'vasc'}

# Añadir columna binaria
df['etiqueta'] = df['dx'].apply(lambda x: 'benignas' if x in clases_benignas else 'malignas')

# Primero separar test (10%)
df_trainval, df_test = train_test_split(
    df, test_size=0.1, stratify=df['etiqueta'], random_state=42
)

# Luego separar val (5% del total, es decir ~5.5% de trainval)
df_train, df_val = train_test_split(
    df_trainval, test_size=0.055, stratify=df_trainval['etiqueta'], random_state=42
)

# Rutas
carpeta_origen = '../data/raw/imagenes'
carpeta_destino = '../data/processed'

# Crear carpetas para train, val y test
for conjunto, df_conjunto in [('train', df_train), ('val', df_val), ('test', df_test)]:
    for clase in ['benignas', 'malignas']:
        carpeta_clase = os.path.join(carpeta_destino, conjunto, clase)
        os.makedirs(carpeta_clase, exist_ok=True)

    for _, fila in df_conjunto.iterrows():
        nombre_imagen = fila['image_id'] + ".jpg"
        etiqueta = fila['etiqueta']
        origen = os.path.join(carpeta_origen, nombre_imagen)
        destino = os.path.join(carpeta_destino, conjunto, etiqueta, nombre_imagen)

        if os.path.exists(origen):
            shutil.copy2(origen, destino)
        else:
            print(f'Imagen no encontrada: {origen}')

print("Split completado: train, val y test creados en ../data/processed")

Split completado: train, val y test creados en ../data/processed


In [ ]:
import pandas as pd
import os
import shutil
from sklearn.model_selection import train_test_split

# Leer el CSV
df = pd.read_csv("../data/raw/HAM10000_metadata.csv")

# Primero separar test (10%)
df_trainval, df_test = train_test_split(
    df, test_size=0.1, stratify=df['dx'], random_state=42
)

# Luego separar val (5% del total, es decir ~5.5% de trainval)
df_train, df_val = train_test_split(
    df_trainval, test_size=0.055, stratify=df_trainval['dx'], random_state=42
)

# Rutas
carpeta_origen = '../data/raw/imagenes'
carpeta_destino = '../data/full'

# Crear carpetas para train, val y test según dx
for conjunto, df_conjunto in [('train', df_train), ('val', df_val), ('test', df_test)]:
    # Crear una carpeta por cada clase dx
    for clase in df['dx'].unique():
        carpeta_clase = os.path.join(carpeta_destino, conjunto, clase)
        os.makedirs(carpeta_clase, exist_ok=True)

    # Copiar imágenes en la carpeta correspondiente a su dx
    for _, fila in df_conjunto.iterrows():
        nombre_imagen = fila['image_id'] + ".jpg"
        origen = os.path.join(carpeta_origen, nombre_imagen)
        destino = os.path.join(carpeta_destino, conjunto, fila['dx'], nombre_imagen)

        if os.path.exists(origen):
            shutil.copy2(origen, destino)
        else:
            print(f'Imagen no encontrada: {origen}')

print("Split completado: train, val y test creados en ../data/full, organizados por dx")


Split completado: train, val y test creados en ../data/processed, organizados por dx


Colocar malignas en una sola carpeta

In [9]:
def preparar_dataset_malignas(metadata_path="../data/raw/HAM10000_metadata.csv",
                              carpeta_origen="../data/raw/imagenes",
                              carpeta_destino="../data/malignas_classes",
                              test_size=0.1,
                              val_size=0.05,
                              random_state=42):
    # Leer metadata
    df = pd.read_csv(metadata_path)

    # Definir clases benignas y malignas
    clases_benignas = {'nv', 'bkl', 'df', 'vasc'}
    clases_malignas = set(df['dx'].unique()) - clases_benignas

    # Añadir columna binaria
    df['etiqueta'] = df['dx'].apply(lambda x: 'benignas' if x in clases_benignas else 'malignas')

    # Primero separar test
    df_trainval, df_test = train_test_split(
        df, test_size=test_size, stratify=df['etiqueta'], random_state=random_state
    )

    # Luego separar val del trainval
    df_train, df_val = train_test_split(
        df_trainval, test_size=val_size, stratify=df_trainval['etiqueta'], random_state=random_state
    )

    # Crear carpetas destino
    for conjunto, df_conjunto in [('train', df_train), ('val', df_val), ('test', df_test)]:
        # Carpetas para cada clase maligna
        for clase in clases_malignas:
            carpeta_clase = os.path.join(carpeta_destino, conjunto, clase)
            os.makedirs(carpeta_clase, exist_ok=True)

        # Copiar imágenes
        for _, fila in df_conjunto.iterrows():
            nombre_imagen = fila['image_id'] + ".jpg"
            origen = os.path.join(carpeta_origen, nombre_imagen)

            if fila['etiqueta'] != 'benignas':
                destino = os.path.join(carpeta_destino, conjunto, fila['dx'], nombre_imagen)

                if os.path.exists(origen):
                    shutil.copy2(origen, destino)
                else:
                    print(f"Imagen no encontrada: {origen}")

    print("Split completado: train, val y test creados en", carpeta_destino)

# Ejecutar
preparar_dataset_malignas()


Split completado: train, val y test creados en ../data/malignas_classes


### Contar archivos por clase

In [12]:
import os

def contar_archivos_por_clase(directorio_base):
    clases_totales = {}  # acumulador por clase

    for conjunto in ["train", "test",  "val"]:
        ruta_conjunto = os.path.join(directorio_base, conjunto)
        if not os.path.exists(ruta_conjunto):
            print(f"No existe la carpeta: {ruta_conjunto}")
            continue

        print(f"\nConjunto: {conjunto}")
        for carpeta in os.listdir(ruta_conjunto):
            ruta_carpeta = os.path.join(ruta_conjunto, carpeta)
            if os.path.isdir(ruta_carpeta):
                archivos = [f for f in os.listdir(ruta_carpeta) 
                            if os.path.isfile(os.path.join(ruta_carpeta, f))]
                cantidad = len(archivos)
                print(f"  Carpeta: {carpeta} -> {cantidad} archivos")

                # acumular por clase
                if carpeta not in clases_totales:
                    clases_totales[carpeta] = 0
                clases_totales[carpeta] += cantidad

    # Mostrar suma total por clase
    print("\nSuma total por clase (train + test):")
    for clase, total in clases_totales.items():
        print(f"  {clase} -> {total} archivos")

# Ejemplo de uso
directorio_base = "../data/processed"  # Ruta base que contiene train y test
contar_archivos_por_clase(directorio_base)



Conjunto: train
  Carpeta: benignas -> 6855 archivos
  Carpeta: malignas -> 1662 archivos

Conjunto: test
  Carpeta: benignas -> 807 archivos
  Carpeta: malignas -> 195 archivos

Conjunto: val
  Carpeta: benignas -> 399 archivos
  Carpeta: malignas -> 97 archivos

Suma total por clase (train + test):
  benignas -> 8061 archivos
  malignas -> 1954 archivos


In [6]:
import pandas as pd
import os

# Leer el CSV
df = pd.read_csv("../data/raw/HAM10000_metadata.csv")

def contar_por_dx(carpeta, df):
    conteo = {}
    
    # Recorremos los archivos en la carpeta
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".jpg"):
            image_id = archivo.replace(".jpg", "")
            fila = df[df['image_id'] == image_id]
            if not fila.empty:
                dx = fila['dx'].values[0]
                conteo[dx] = conteo.get(dx, 0) + 1
    return conteo

carpeta_test = "../data/processed/test/malignas"
conteo_test = contar_por_dx(carpeta_test, df)

# Calcular total
total = sum(conteo_test.values())

print("Conteo en carpeta test por dx:")
for dx, cantidad in conteo_test.items():
    porcentaje = (cantidad / total) * 100
    print(f"{dx}: {cantidad} ({porcentaje:.2f}%)")

print(f"Total imágenes: {total}")


Conteo en carpeta test por dx:
mel: 111 (56.92%)
bcc: 53 (27.18%)
akiec: 31 (15.90%)
Total imágenes: 195


In [7]:
carpeta_test = "../data/processed/train/malignas"
conteo_test = contar_por_dx(carpeta_test, df)

# Calcular total
total = sum(conteo_test.values())

print("Conteo en carpeta test por dx:")
for dx, cantidad in conteo_test.items():
    porcentaje = (cantidad / total) * 100
    print(f"{dx}: {cantidad} ({porcentaje:.2f}%)")

print(f"Total imágenes: {total}")

Conteo en carpeta test por dx:
mel: 1002 (56.96%)
akiec: 296 (16.83%)
bcc: 461 (26.21%)
Total imágenes: 1759
